<h1>CNNs for EPRV on HARPS</h1>
The goal here is to training a CNN using HARPS images to the outputs of the HARPS EPRV extraction pipeline to see it a large of NN can replicated more explicit modeling.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

ModuleNotFoundError: No module named 'torch'

In [2]:
print(torch.cuda.is_available())

NameError: name 'torch' is not defined

In [3]:
torch.cuda.device_count()

NameError: name 'torch' is not defined

In [4]:
cuda = torch.device('cuda')
a = torch.tensor([1., 2.], device=cuda)

NameError: name 'torch' is not defined

In [5]:
import subprocess
subprocess.run(["python", "--version"])

Python 3.7.12


CompletedProcess(args=['python', '--version'], returncode=0)

In [8]:
import torchvision

ModuleNotFoundError: No module named 'torchvision'

In [11]:
subprocess.run(['python','--version'])

Python 3.7.12


CompletedProcess(args=['python', '--version'], returncode=0)

<h2>Model definition</h2>

Higher stride at higher layers,

In [9]:
class DownSizeNet(nn.Module):
    def __init__(self, in_size, out_size, normalize=True, leaky_slope=0.2):
        super(DownSizeNet, self).__init__()
        layers = [nn.Conv2d(in_size, out_size, 3, stride=1, padding=1, bias=False).double()]
        if normalize:
            layers.append(nn.BatchNorm2d(out_size, 0.8).double())
        layers.append(nn.LeakyReLU(leaky_slope).double())
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# class DenseLayer(nn.Module):
#     def __init__(self):
#         super(DenseLayer)
    
class RV_Model(nn.Module):
    def __init__(self):
        super(RV_Model, self).__init__()
        self.down1 = DownSizeNet(3, 64, normalize=False)
        self.down2 = DownSizeNet(64, 128)
        self.down3 = DownSizeNet(128, 256)
        self.down4 = DownSizeNet(256, 512)
        self.down5 = DownSizeNet(512, 512)
        self.down6 = DownSizeNet(512, 512)
        self.final = nn.Sequential(
            nn.Conv2d(512, 1, 3, stride=1, padding=1).double(), nn.Tanh().double()
        )
        
        
    def forward(self, x):
        # Propogate noise through fc layer and reshape to img shape
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)
        d6 = self.down6(d5)
        
        return self.final(d6)

In [10]:
from torch.utils.data import Dataset
import glob

In [11]:
from astropy.io import fits

<h2>Dataset importing</h2>
The question here is how is the data organized in the directory and how can it be imported with the target RV. 

Not the OG data but after the data is saved from the pre processing step.

In [12]:
import pickle

In [21]:
import numpy as np
import h5py as h5
filename = "../data/HARPS/PEG51/51Peg_1020-1546_1530-2056.h5"

# ds = h5.File(filename,'r')

ModuleNotFoundError: No module named 'h5py'

In [ ]:
def get_length(ds):
    total = 0
    address_book = []
    rvs = []
    
    for visit_name,vist_info in ds['visits'].items():
        for hdu_num in ds['images'][visit_name].keys():
            value = np.sum(np.array(ds['images'][visit_name][hdu_num]).shape)
            if value != 526*2:
                print(value)
            if value == (526*2):
                address_book.append({'sci': visit_name,
                                     'flat': '',
                                     'cali': '',
                                     'hdu': hdu_num})
                rvs.append(np.double(vist_info.attrs['ESO DRS CCF RVC']))
                flat_bool = False
                cali_bool = False
                for key in vist_info.attrs.keys():
                    if vist_info.attrs[key] == 'FLAT' and not flat_bool:
                        address_book[-1]['flat'] = key
                        flat_bool = True
                    if vist_info.attrs[key] == 'THAR_THAR' and not cali_bool:
                        address_book[-1]['cali'] = key
                        cali_bool = True
                
                total += 1
    return total, address_book, rvs
# length, address_book, rvs = get_length(ds)

In [ ]:
a = [1,2]
b = [3,4]
a += b
print(a)

In [ ]:
class Multi_H5_Dataset(Dataset):
    def __init__(self, h5_files):
        self.h5_files = h5_files
        self.type = torch.Tensor
#         self.ds = {}
        self.length = 0
        self.rvs = np.array([])
        self.address = []
        for file in h5_files:
            ds = h5.File(file,'r')
#             self.ds[file] = ds
            length, address, rvs = get_length(ds)
            for addy in address:
                addy['file'] = file
            self.address += address
            self.length += length
            self.rvs = np.hstack((self.rvs,np.array(rvs)))
        print(self.length,len(self.rvs))

    def __getitem__(self, index):
        ds = h5.File(self.address[index]['file'],'r')
        
        image = np.stack((np.array(ds['images'][self.address[index]['sci']][self.address[index]['hdu']])
                         ,np.array(ds['images'][self.address[index]['flat']][self.address[index]['hdu']])
                         ,np.array(ds['images'][self.address[index]['cali']][self.address[index]['hdu']])),axis=0)
        return {'img': self.type(image).double(), 'rvs': self.rvs[index]}
    
    
    def __len__(self):
        return self.length

In [ ]:
torch.cuda.is_available()

In [ ]:
import cudatoolkit

In [ ]:
files1 = glob.glob('/scratch/mdd423/CNN_EPRV/data/HARPS/PEG51/*.h5')
# files2 = glob.glob('/scratch/mdd423/CNN_EPRV/data/HARPS/PEG51/51Peg-2/*.h5')
# files3 = glob.glob('/scratch/mdd423/CNN_EPRV/data/HARPS/PEG51/51Peg-3/*.h5')
# files4 = glob.glob('/scratch/mdd423/CNN_EPRV/data/HARPS/PEG51/51Peg-4/*.h5')
# file_list = [*files1,*files2,*files3,*files4]

In [ ]:
# file_list

In [ ]:
dataset = Multi_H5_Dataset(files1)

In [ ]:
class RV_Dataset(Dataset):
    def __init__(self, h5_file):
        self.h5_file = h5_file
        self.type = torch.Tensor
        self.ds = h5.File(h5_file,'r')
        self.length, self.address, self.rvs = get_length(self.ds)
        print(self.length,len(self.rvs))

    def __getitem__(self, index):
        
        image = np.stack((np.array(self.ds['images'][self.address[index][0]][self.address[index][-1]])
                         ,np.array(self.ds['images'][self.address[index][1]][self.address[index][-1]])
                         ,np.array(self.ds['images'][self.address[index][2]][self.address[index][-1]])),axis=0)
        return {'img': self.type(image).double(), 'rvs': self.rvs[index]}
    
    
    def __len__(self):
        return self.length


In [14]:
import os.path

<h2>Defining Fitting Process</h2>
including hyperparameters, the loss function, and the optimization algo

In [15]:
lr = 0.001
b1 = 0.9
b2 = 0.999

In [16]:
model = RV_Model()
mse_loss = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(b1, b2))

In [17]:
# directory = something
batch_size = 2
n_cpu = 1
dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=n_cpu,
)

NameError: name 'dataset' is not defined

In [18]:
import time
import datetime

<h2>Training Step</h2>
the working step!

In [19]:
import sys

In [22]:
prev_time = time.time()
n_epochs = 1
# validiter = iter(validloader)
for epoch in range(n_epochs):
    for i, batch in enumerate(dataloader):
        
        optimizer.zero_grad()
        something = model(batch['img'])
        y         = something.view(batch['img' t].shape[0], -1).median(1)
        loss      = mse_loss(y,batch['rvs'].double())
        loss.backward()
        optimizer.step()
    
        sys.stdout.write(
                "\r[Epoch %d/%d] [Batch %d/%d] [Loss: %f]"
                % (
                    epoch,
                    n_epochs,
                    i,
                    len(dataloader),
                    loss.item()
                )  
        )        

NameError: name 'dataloader' is not defined